# Environment

In [1]:
# !pip install --upgrade pip

In [1]:
# !pip install seaborn
!pip install multiprocessing-logging
!pip install spacy
!pip install gensim
!pip install nltk
!pip install psutil
!pip install psycopg2
!pip install pymorphy2

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: http://mirror.ca.sbrf.ru/pypi/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: http://mirror.ca.sbrf.ru/pypi/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: http://mirror.ca.sbrf.ru/pypi/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: http://mirror.ca.sbrf.ru/pypi/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: http://mirror.ca.sbrf.ru/pypi/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: http://mirror.ca.sbrf.ru/pypi/simple
     |████████████████████████████████| 379 kB 37.8 MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /opt/python/virtualenv/jupyter/bin/python -c 'import io,

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [4]:
# Basic
import os
import re
import sys
import glob
import gc
import pickle
import psutil
import psycopg2
import datetime
import numpy as np
import pandas as pd

from pathlib import Path
from pprint import pprint
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict, namedtuple, deque, Counter
from typing import (List, Dict, Any, NoReturn, 
                    Tuple, Optional, Union)
from tqdm import tqdm
from datetime import datetime as dt
from sklearn.model_selection import train_test_split

import logging
import multiprocessing
from multiprocessing_logging import install_mp_handler
logging.basicConfig(level=logging.DEBUG,
                   format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

import warnings
warnings.filterwarnings('ignore')

In [5]:
# W2V
import spacy
import pymorphy2
from gensim.models import KeyedVectors
from gensim.models.phrases import Phrases
from nltk.stem.snowball import SnowballStemmer
from gensim.models.phrases import Phrases, npmi_scorer
from gensim.models import word2vec, keyedvectors

2021-07-15 16:47:59,785 - tensorflow - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.


In [6]:
import logging
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)

## Define paths

In [7]:
print(f"CPU count: {psutil.cpu_count()}")
print(f"CPU utilization: {psutil.cpu_percent()}%")
mem_stats = psutil.virtual_memory()
print(f"Memory total: {mem_stats.total / 1048576} MB.")
print(f"Memory available: {mem_stats.available / 1048576} MB.")
print(f"Memory used: {mem_stats.used / 1048576} MB.")

CPU count: 8
CPU utilization: 2.8%
Memory total: 64265.0703125 MB.
Memory available: 23256.09765625 MB.
Memory used: 39635.5 MB.


In [8]:
BASE_DIR = Path("..")
DATA_DIR  = BASE_DIR / "data"
MODEL_DIR  = BASE_DIR / "models"

## Load data

In [9]:
def load_data(start_date: str, end_date: str, is_prediction: Tuple[bool], 
              maxnum: int=1e6):
    
    print(f"Creating connection to DB...")
    connection = psycopg2.connect("dbname='{}' user='{}' host='{}' password='{}' port = {}".format(
        'pgarmat', 'postgres', 'tklis-sm0000002.vm.mos.cloud.sbrf.ru', 'ZxqZo8lX', '5432'))
    print(f"Connection created.")
    
    with connection.cursor() as cursor:
        try:
            cursor.execute(f"""
                select
                req_num, 
                msg, 
                incident,
                incident_probability,
                req_reg_datetime, 
                product, 
                subject,
                s_subject, 
                subproduct,
                grp, 
                channel
                from ds70.request_messages
                where 1=1
                --and client_msg_vector is null 
                and is_prediction in {is_prediction}
                and req_reg_datetime between '{start_date}' and '{end_date}'
                limit {maxnum};""")
            requests = cursor.fetchall()
        except Exception as ee:
            print(ee)
            connection.rollback()
            
        data = pd.DataFrame(requests, columns=["req_num",
                                               "msg",
                                               "incident",
                                               "incident_probability",
                                               "req_reg_datetime",
                                               "product",
                                               "subject",
                                               "s_subject",
                                               "subproduct",
                                               "grp",
                                               "channel"])
        print(f"Loaded data shape: {data.shape} and size: {sys.getsizeof(data) / 1048576} MB.")
        return data

In [10]:
def update_data_vectors(req_nums: List[str], vectors: List[np.ndarray], col: str):
   
    print(f"Creating connection to DB...")
    connection = psycopg2.connect("dbname='{}' user='{}' host='{}' password='{}' port = {}".format(
        'pgarmat', 'postgres', 'tklis-sm0000002.vm.mos.cloud.sbrf.ru', 'ZxqZo8lX', '5432'))
    print(f"Connection created.")
   
    for rnum, vec in zip(req_nums, vectors):
        with connection.cursor() as cursor:
            try:
                cursor.execute(f"""
                    UPDATE ds70.request_messages
                    SET {col}=ARRAY{vec}::integer[]
                    where 1=1
                    and req_num = '{rnum}';""")
                connection.commit()
            except Exception as ee:
                print(ee)
                connection.rollback()
                
                
                
def update_datatime_db(req_nums: List[str], curr_dt: datetime.datetime, col: str):
   
    print(f"Creating connection to DB...")
    connection = psycopg2.connect("dbname='{}' user='{}' host='{}' password='{}' port = {}".format(
        'pgarmat', 'postgres', 'tklis-sm0000002.vm.mos.cloud.sbrf.ru', 'ZxqZo8lX', '5432'))
    print(f"Connection created.")
    with connection.cursor() as cursor:
        try:
            cursor.execute(f"""
                    UPDATE ds70.request_messages
                    SET {col}='{curr_dt}'
                    where 1=1
                    and req_num in {tuple(req_nums)};""")
            connection.commit()
        except Exception as ee:
            print(ee)
            connection.rollback()

In [11]:
def load_vectors_data(start_date: str, end_date: str, is_prediction: Tuple[bool], 
                      maxnum: int=1e6):
    
    print(f"Creating connection to DB...")
    connection = psycopg2.connect("dbname='{}' user='{}' host='{}' password='{}' port = {}".format(
        'pgarmat', 'postgres', 'tklis-sm0000002.vm.mos.cloud.sbrf.ru', 'ZxqZo8lX', '5432'))
    print(f"Connection created.")
    
    with connection.cursor() as cursor:
        try:
            cursor.execute(f"""
                select
                req_num, 
                req_reg_datetime, 
                msg,
                product, 
                subject,
                s_subject, 
                subproduct,
                client_msg_vector
                from ds70.request_messages
                where 1=1
                and client_msg_vector is not null
                and is_prediction in {is_prediction}
                and req_reg_datetime between '{start_date}' and '{end_date}'
                limit {maxnum};""")
            requests = cursor.fetchall()
        except Exception as ee:
            print(ee)
            connection.rollback()
            
        data = pd.DataFrame(requests, columns=["req_num",
                                               "req_reg_datetime",
                                               "msg",
                                               "product",
                                               "subject",
                                               "s_subject",
                                               "subproduct",
                                               "client_msg_vector"])
        print(f"Loaded data shape: {data.shape} and size: {sys.getsizeof(data) / 1048576} MB.")
        return data

In [12]:
df = load_data(start_date='2021-02-01 0:00:01', end_date='2021-02-10 23:59:59', is_prediction=(True, False),
               maxnum=600_000)
# test_df = load_vectors_data(start_date='2021-03-01 0:00:01', end_date='2021-03-30 23:59:59', is_prediction=(True, False),
#                           maxnum=200_000)

Creating connection to DB...
Connection created.
Loaded data shape: (600000, 11) and size: 2259.2678871154785 MB.


In [13]:
df['by_role'] = df.msg.str.contains('operator')
print(df['by_role'].value_counts())

df1 = df.copy().loc[df['by_role']]
print(df1.shape[0])

True     595371
False      4629
Name: by_role, dtype: int64
595371


## Configure preprocessing

In [15]:
def file_opener(filename: str) -> str:
    with open(filename, 'rt', encoding='utf-8-sig') as src:
        file = src.read()
    return "|".join([x for x in file.split('\n') if x])


class DataPreprocessorLemmatizer:

    text_features = ['subject', 's_subject', 'product',
                     'subproduct', 'grp', 'channel', 'msg']
    stopgrams = [
             #'ADVB',   # наречие -> почему, онлайн, сегодня, завтра, вообще
             'CONJ',   # союз
             'PRCL',   # частица
             'PRED',   # предикатив
             'NPRO',   # местоимение-сущ.
             'INTJ',   # междометие
             #'Abbr',   # аббревиатура -> смс, втб, мтс 
             'Erro',   # ошибка
             'Dist',   # искажение
             'Ques',   # вопросительное слово
             'Dmns',   # указательное слово
             'Prnt'   # вводное слово
            ]

    def __init__(self, only_unique: bool=True, intro_words_path: str = './lists'):
        self.__morph = pymorphy2.MorphAnalyzer()
        self.__nlp = spacy.blank('ru')
        self.__only_unique = only_unique
        self.__intro_words = file_opener(os.path.join(intro_words_path, 'intro.txt'))
        self.__nltk_stopwords = file_opener(os.path.join(intro_words_path, 'NLTK_stopwords.txt')).split("|")

    def get_stopwords(self):
        """ 
        Check intro-words list.
        """
        return self.__nltk_stopwords

    def get_intro_words(self):
        """ 
        Check intro-words list.
        """
        return self.__intro_words
    
    def get_analyzer(self):
        """
        Allow to access to Pymorphy Analyzer instance.
        """
        return self.__morph

    def _reformat_datetime(self, dt: datetime) -> str:
        """ Reformat datetime and convert to string """
        return dt.strftime("%Y-%m-%d %H-%M-%S")
    
    def _process_text(self, text: str):
        """ 
        Process single text and return list of tokens.
        """
        if pd.isna(text):
            return []
       # Pre-processing part 
        text = [str(token).lower()
                for token in self.__nlp.make_doc(text)
                if (token and token.is_alpha and len(str(token.text)) > 2 and ~token.is_stop)]
        # Processing part
        clean_text = []
        for token in text:
            token = self.__morph.parse(str(token).lower())[0]
            if ((token.normal_form not in self.__nltk_stopwords) 
                and (token.normal_form not in self.__intro_words)
                and all([tag not in token.tag for tag in self.stopgrams])):
                clean_text.append(token.normal_form)
        if self.__only_unique:
            clean_text = list(set(clean_text))
        return clean_text
    

    def process_texts(self, texts: List[str]):
        """ 
        Process list of texts and return list of lists of tokens.
        """
        return [self._process_text(text) for text in tqdm(texts)]


    def process(self, data: pd.DataFrame,
                features_cols: Optional[List[str]] = None, copy: bool=True) -> pd.DataFrame:
        """
        Preprocess text for language modelling.
         - clean introduction words, numbers and small prefixes;
         - tokenize and lemmatize texts;
        """
        print("[INFO] Text processing...")
        if not features_cols:
            features_cols = self.text_features

        for col_name in features_cols:
            print(f"[INFO] Process '{col_name}' column...")
            data_processed = self.process_texts(data[col_name].fillna("").to_list())
            if not copy:
                data[col_name] = data_processed
            else:
                data[col_name + "_proc"] = data_processed

        print("[INFO] Text finished.")
        return data

In [16]:
class DataPreprocessorLemmatizerMultiProc:

    text_features = ['subject', 's_subject', 'product',
                     'subproduct', 'grp', 'channel', 'msg']
    stopgrams = [
             #'ADVB',   # наречие -> почему, онлайн, сегодня, завтра, вообще
             'CONJ',   # союз
             'PRCL',   # частица
             'PRED',   # предикатив
             'NPRO',   # местоимение-сущ.
             'INTJ',   # междометие
             #'Abbr',   # аббревиатура -> смс, втб, мтс 
             'Erro',   # ошибка
             'Dist',   # искажение
             'Ques',   # вопросительное слово
             'Dmns',   # указательное слово
             'Prnt'   # вводное слово
            ]

    def __init__(self, morph: pymorphy2.MorphAnalyzer, nlp: spacy.Language,
                 num_processors: int=16, chunksize: int=100, 
                 only_unique: bool=True, 
                 intro_words_path: str = './lists'):
        self.__num_processors = num_processors
        self.__chunksize = chunksize
        self.__morph = morph
        self.__nlp = nlp
        self.__only_unique = only_unique
        self.__intro_words = file_opener(os.path.join(intro_words_path, 'intro.txt'))
        self.__nltk_stopwords = file_opener(os.path.join(intro_words_path, 'NLTK_stopwords.txt')).split("|")

    def get_stopwords(self):
        """ 
        Check intro-words list.
        """
        return self.__nltk_stopwords

    def get_intro_words(self):
        """ 
        Check intro-words list.
        """
        return self.__intro_words
    
    def get_analyzer(self):
        """
        Allow to access to Pymorphy Analyzer instance.
        """
        return self.__morph

    def _reformat_datetime(self, dt: datetime) -> str:
        """ Reformat datetime and convert to string """
        return dt.strftime("%Y-%m-%d %H-%M-%S")
    
    def _process_text(self, text: str):
        """ 
        Process single text and return list of tokens.
        """
        if pd.isna(text):
            return []
       # Pre-processing part 
        text = [str(token).lower()
                for token in self.__nlp.make_doc(text)
                if (token and token.is_alpha and len(str(token.text)) > 2 and ~token.is_stop)]
        # Processing part
        clean_text = []
        for token in text:
            token = self.__morph.parse(str(token).lower())[0]
            if ((token.normal_form not in self.__nltk_stopwords) 
                and (token.normal_form not in self.__intro_words)
                and all([tag not in token.tag for tag in self.stopgrams])):
                clean_text.append(token.normal_form)
        if self.__only_unique:
            clean_text = list(set(clean_text))
        return clean_text
    

    def process_texts(self, texts: List[str]):
        """ 
        Process list of texts and return list of lists of tokens.
        """
        with multiprocessing.Pool(self.__num_processors) as pool:
            processed_texts = pool.map(self._process_text, texts, chunksize=self.__chunksize)
        return processed_texts


    def process(self, data: pd.DataFrame,
                features_cols: Optional[List[str]] = None, copy: bool=True) -> pd.DataFrame:
        """
        Preprocess text for language modelling.
         - clean introduction words, numbers and small prefixes;
         - tokenize and lemmatize texts;
        """
        print("[INFO] Text processing...")
        if not features_cols:
            features_cols = self.text_features

        for col_name in features_cols:
            print(f"[INFO] Process '{col_name}' column...")
            data_processed = self.process_texts(data[col_name].fillna("").to_list())
            if not copy:
                data[col_name] = data_processed
            else:
                data[col_name + "_proc"] = data_processed

        print("[INFO] Text finished.")
        return data

### Parallel processing

In [17]:
print(f"Number CPU: {multiprocessing.cpu_count()}")

Number CPU: 8


In [18]:
morph = pymorphy2.MorphAnalyzer()
nlp = spacy.blank('ru')
only_unique = False
intro_words = file_opener(os.path.join(str(BASE_DIR / 'lists'), 'intro.txt')).lower().split("|")
nltk_stopwords = file_opener(os.path.join(str(BASE_DIR / 'lists'), 'NLTK_stopwords.txt')).split("|")

install_mp_handler()

stopgrams = [
             'CONJ',   # союз
             'PRCL',   # частица
             'PRED',   # предикатив
             'NPRO',   # местоимение-сущ.
             'INTJ',   # междометие
             'Erro',   # ошибка
             'Dist',   # искажение
             'Ques',   # вопросительное слово
             'Dmns',   # указательное слово
             'Prnt'   # вводное слово
            ]

def process_texts(text):
    """ 
    Process single text and return list of tokens.
    """
    if pd.isna(text):
        return []
    # Pre-processing part 
    text = [str(token).lower()
            for token in nlp.make_doc(text)
            if (token and token.is_alpha and len(str(token.text)) > 2 and ~token.is_stop)]
    # Processing part
    clean_text = []
    for token in text:
        token = morph.parse(str(token).lower())[0]
        if ((token.normal_form not in nltk_stopwords) and (token.normal_form not in intro_words)
            and all([tag not in token.tag for tag in stopgrams])):
            clean_text.append(token.normal_form)
    if only_unique:
        clean_text = list(set(clean_text))
    return clean_text

2021-07-15 16:56:09,173 - pymorphy2.opencorpora_dict.wrapper - INFO - Loading dictionaries from /home/abdullaeva2-ia_ca-sbrf-ru/.local/lib/python3.8/site-packages/pymorphy2_dicts_ru/data
2021-07-15 16:56:10,478 - pymorphy2.opencorpora_dict.wrapper - INFO - format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [23]:
class DataVectorizer:
 
    def __init__(self, phrase_model: Phrases = None,
                 w2v_vectorizer: Optional[KeyedVectors] = None):
        """
        Need to load pre-trained models from files.
        """
        self._w2v = w2v_vectorizer
        self._phrase_model = phrase_model
        self._vocab_size, self._embedding_size = self._w2v.vectors.shape
 
       
    @classmethod
    def load(cls, save_path: str, phrases_fn: str, w2v_fn: str) -> object:
        """
        Load pre-trained models from files and init.
        """
        phrase_model = Phrases.load(os.path.join(save_path, phrases_fn))
        w2v_vectorizer = KeyedVectors.load_word2vec_format(os.path.join(save_path, w2v_fn), binary=False)
        return cls(phrase_model=phrase_model,
                   w2v_vectorizer=w2v_vectorizer)
   
    
    def _word2id(self, token_seq: List[List[str]], max_len: int):
        """
        Convert token to id in W2V vocabulary.
        """
        id_seq = [[self._w2v.get_index(tok) + 1 for tok in doc if self._w2v.has_index_for(tok)]
                  for doc in tqdm(token_seq)]
#         id_seq = pad_sequences(id_seq, value=0, maxlen=max_len, padding='post', truncating='post')
        return id_seq
 
   
    def vectorize(self, data: List[str], max_len: int) -> List[int]:
        """
        Vectorize data:
            - encode words as ids
            - reduce dimensions
        """
        data = [self._phrase_model[d] if len(d) > 0 else np.zeros((max_len,)) for d in data]
        data = self._word2id(data, max_len)
        return data
   
    def get_vectors(self):
        return self._w2v.vectors

In [17]:
# processor = DataPreprocessorLemmatizer(intro_words_path=str(BASE_DIR / 'lists'))

### Select only clients messages

In [20]:
def select_role_speech(df: pd.DataFrame, msg_col: str, role: str):
    if role == 'client':
        client_pattern = re.compile(f"(client:)([А-Яа-яЁё#\s]+)(operator:)?")
        df['client_msg'] = df[msg_col].apply(lambda x: " ".join([i[1].strip() for i in client_pattern.findall(x)]))
    else:
        operator_pattern = re.compile(f"(operator:)([А-Яа-яЁё#\s]+)(client:)?")
        df['operator_msg'] = df[msg_col].apply(lambda x: " ".join([i[1].strip() for i in operator_pattern.findall(x)]))
    return df

df1 = select_role_speech(df1, 'msg', role='client')

In [22]:
df1.head()

,req_num,msg,incident,incident_probability,req_reg_datetime,product,subject,s_subject,subproduct,grp,channel,by_role,client_msg
0,2102010224893100.0,client:здравствуйте звоню вам по такому поводу...,None,NaN,2021-02-01 01:35:09,Кредиты,Плановое/досрочное погашение,Консультация/Разъяснения по структуре платежа ...,Жилищный кредит,Консультация,Не определено,True,здравствуйте звоню вам по такому поводу как бы...
1,2102010224893400.0,operator: #ФИО здравствуйте client:здравствуй...,None,NaN,2021-02-01 01:35:09,Тематики ЕСЗК,Обращения НЕ по продуктам и услугам банка,Обращение/вопрос не по продуктам и услугам Банка,Проблемное обращение,Транзит,ЕРКЦ,True,здравствуйте почему я не могу соединиться со с...
2,2102010224894200.0,operator: #ФИО здравствуйте client:здравствуй...,None,NaN,2021-02-01 01:35:13,Кредиты,Сопровождение действующего кредита,"Сопровождение кредита (в тч ведение договора, ...",Жилищный кредит,Сервисный запрос,Не определено,True,здравствуйте я хотел бы вот узнать я в двадцат...
3,2102010224894600.0,operator:здравствуйте client:здравствуйте деву...,None,NaN,2021-02-01 01:35:15,Банковская карта,Предоставление информации,Тарифы/лимиты/ условия предоставления и обслуж...,Дебетовая карта,Консультация,Не определено,True,здравствуйте девушка подскажите пожалуйста вот...
4,2102010224896300.0,operator:меня зовут #ФИО здравствуйте client...,None,NaN,2021-02-01 01:35:25,Банковская карта,Подтверждение рисковой операции,Сверхлимитная операция,Дебетовая карта,Жалоба,СБОЛ,True,здравствуйте можно мне подтвердить платеж #ЧИС...


### Process text

In [23]:
# print(df1.shape)
# n_to_train = 200_000
# df_train = df1.iloc[:n_to_train]
print("Training data shape:", df1.shape)

Training data shape: (595371, 13)


In [ ]:
# morph = pymorphy2.MorphAnalyzer()
# nlp = spacy.blank('ru')
# processor = DataPreprocessorLemmatizerMultiProc(morph=morph, nlp=nlp,
#                                                 num_processors=32, chunksize=10, intro_words_path=str(BASE_DIR / 'lists'))

In [ ]:
# %%time
# _ = processor.process(data=sample_df, features_cols=['msg'])

In [ ]:
%%time
NUMPROCS = multiprocessing.cpu_count() - 1

install_mp_handler()
with multiprocessing.Pool(NUMPROCS, initializer=install_mp_handler) as pool:
    processed_data = pool.map(process_texts, df1['client_msg'].fillna("").to_list(), chunksize=200)

In [25]:
df1['msg_proc'] = [txt for txt in processed_data]
print(len(processed_data))
processed_data[:2]

595371


[['хотеть',
  'уточнить',
  'вчера',
  'карта',
  'первый',
  'январь',
  'вечером',
  'списать',
  'деньга',
  'описание',
  'стоить',
  'сбербанк',
  'онлайн',
  'скб',
  'банк',
  'хотеть',
  'узнать',
  'снять',
  'номер',
  'карта',
  'назвать',
  'полностью',
  'секундочка',
  'фамилия',
  'имя',
  'отчество',
  'полностью',
  'надежда',
  'кодовый',
  'слово',
  'помнить',
  'кодовый',
  'слово',
  'ваш',
  'помнить',
  'сумма',
  'ещё',
  'уточнить',
  'сумма',
  'ещё',
  'уточнить',
  'сумма',
  'первый',
  'вечером',
  'час',
  'минута',
  'сайт',
  'видимо',
  'банк',
  'перевод',
  'перевод',
  'сайт',
  'банк',
  'дать',
  'карта',
  'указываться',
  'перевод',
  'проходить',
  'перевод',
  'никакой',
  'операция',
  'совершать',
  'делать',
  'совершать',
  'перевод',
  'понять',
  'случай',
  'действие',
  'ваш',
  'карта',
  'приостановить',
  'дальнейший',
  'консультация',
  'соединить',
  'специалист',
  'ожидание',
  'длительный',
  'удобно',
  'удобно',
  'слышать'

In [26]:
print(df1.shape)
df1.head(2)

(595371, 14)


,req_num,msg,incident,incident_probability,req_reg_datetime,product,subject,s_subject,subproduct,grp,channel,by_role,client_msg,msg_proc
0,2102010302824400,operator: #ФИО здравствуйте client:здравствуй...,None,NaN,2021-02-01 18:30:28,Банковская карта,Блокировка счета/карты,Блокировка карты,Банковская карта,Внутренний запрос,ЕРКЦ,True,здравствуйте я бы хотела уточнить у меня вчера...,"[хотеть, уточнить, вчера, карта, первый, январ..."
1,2102010302231100.0,operator: #ФИО здравствуйте client:здравствуй...,None,NaN,2021-02-01 18:23:53,Подозрение на мошенничество,Попытка мошенничества без финансовых потерь кл...,Консультация при попытке мошенничества,Подозрение на мошенничество,Консультация,Не определено,True,здравствуйте девушка мне уже в который раз зво...,"[девушка, который, звонить, представляться, сл..."


### Update text vectors

**(if needed)**

In [27]:
dvect = DataVectorizer.load(str(MODEL_DIR), phrases_fn="phrases_gensim_lemmas_12_07_2021.pkl",
                           w2v_fn='word2vec_gensim_lemmas_12_07_2021')

2021-07-14 11:39:21,775 - gensim.utils - INFO - loading Phrases object from ../models/phrases_gensim_lemmas_12_07_2021.pkl
2021-07-14 11:39:21,776 - smart_open.smart_open_lib - DEBUG - {'uri': '../models/phrases_gensim_lemmas_12_07_2021.pkl', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'transport_params': None}
2021-07-14 11:39:25,859 - gensim.utils - INFO - Phrases lifecycle event {'fname': '../models/phrases_gensim_lemmas_12_07_2021.pkl', 'datetime': '2021-07-14T11:39:25.807348', 'gensim': '4.0.1', 'python': '3.8.3 (default, May 19 2020, 18:47:26) \n[GCC 7.3.0]', 'platform': 'Linux-3.10.0-1160.25.1.el7.x86_64-x86_64-with-glibc2.10', 'event': 'loaded'}
2021-07-14 11:39:25,860 - gensim.models.phrases - WARNING - older version of Phrases loaded without common_terms attribute, setting connector_words to an empty set
2021-07-14 11:39:25,860 - gensim.models.keyedvectors - INFO - loading projection w

In [28]:
batch_size = 200_000
for i in range(int(np.ceil(df1.shape[0] // batch_size)) + 1):
    batch = df1.iloc[i*batch_size : (i+1)*batch_size]
    batch = batch.loc[batch["msg_proc"].notnull()]
    print(f"Batch #{i} with {batch.shape[0]}")
    text_vectors = dvect.vectorize(batch["msg_proc"].to_list(), max_len=150)
    batch['token_ids'] = list(map(tuple, text_vectors))
    batch['token_ids'] = batch.token_ids.apply(lambda x: str(list(map(float, x))))
    update_data_vectors(batch.req_num.to_list(), batch.token_ids.to_list(), 'client_msg_vector')
    del batch
    _ = gc.collect()

2021-07-14 11:39:49,623 - gensim.utils - INFO - KeyedVectors lifecycle event {'msg': 'loaded (95552, 200) matrix of type float32 from ../models/word2vec_gensim_lemmas_12_07_2021', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-07-14T11:39:49.623485', 'gensim': '4.0.1', 'python': '3.8.3 (default, May 19 2020, 18:47:26) \n[GCC 7.3.0]', 'platform': 'Linux-3.10.0-1160.25.1.el7.x86_64-x86_64-with-glibc2.10', 'event': 'load_word2vec_format'}


Batch #0 with 200000


100%|██████████| 200000/200000 [00:18<00:00, 11089.02it/s]


Creating connection to DB...
Connection created.
Batch #1 with 200000


100%|██████████| 200000/200000 [00:20<00:00, 9854.95it/s] 


Creating connection to DB...
Connection created.
Batch #2 with 195371


100%|██████████| 195371/195371 [00:20<00:00, 9481.92it/s] 


Creating connection to DB...
Connection created.


### Set current update date-time

In [29]:
update_datatime_db(df1.req_num.to_list(), datetime.datetime.now(), 'vector_update_datetime')

Creating connection to DB...
Connection created.


# Train Vectorizers

## Train Word2Vec

In [33]:
text_corpus = df1.msg_proc.to_list()
print(len(text_corpus))
text_corpus[:10]

595371


[['хотеть',
  'уточнить',
  'вчера',
  'карта',
  'первый',
  'январь',
  'вечером',
  'списать',
  'деньга',
  'описание',
  'стоить',
  'сбербанк',
  'онлайн',
  'скб',
  'банк',
  'хотеть',
  'узнать',
  'снять',
  'номер',
  'карта',
  'назвать',
  'полностью',
  'секундочка',
  'фамилия',
  'имя',
  'отчество',
  'полностью',
  'надежда',
  'кодовый',
  'слово',
  'помнить',
  'кодовый',
  'слово',
  'ваш',
  'помнить',
  'сумма',
  'ещё',
  'уточнить',
  'сумма',
  'ещё',
  'уточнить',
  'сумма',
  'первый',
  'вечером',
  'час',
  'минута',
  'сайт',
  'видимо',
  'банк',
  'перевод',
  'перевод',
  'сайт',
  'банк',
  'дать',
  'карта',
  'указываться',
  'перевод',
  'проходить',
  'перевод',
  'никакой',
  'операция',
  'совершать',
  'делать',
  'совершать',
  'перевод',
  'понять',
  'случай',
  'действие',
  'ваш',
  'карта',
  'приостановить',
  'дальнейший',
  'консультация',
  'соединить',
  'специалист',
  'ожидание',
  'длительный',
  'удобно',
  'удобно',
  'слышать'

In [34]:
%%time
phrase_model = Phrases(text_corpus, min_count=5, threshold=0.3, scoring=npmi_scorer)
text_corpus = [phrase_model[text] for text in text_corpus]

2021-07-14 13:42:59,651 - gensim.models.phrases - INFO - collecting all words and their counts
2021-07-14 13:42:59,651 - gensim.models.phrases - INFO - PROGRESS: at sentence #0, processed 0 words and 0 word types
2021-07-14 13:43:01,293 - gensim.models.phrases - INFO - PROGRESS: at sentence #10000, processed 934902 words and 296756 word types
2021-07-14 13:43:03,010 - gensim.models.phrases - INFO - PROGRESS: at sentence #20000, processed 1867535 words and 471354 word types
2021-07-14 13:43:04,800 - gensim.models.phrases - INFO - PROGRESS: at sentence #30000, processed 2792263 words and 612637 word types
2021-07-14 13:43:06,623 - gensim.models.phrases - INFO - PROGRESS: at sentence #40000, processed 3707834 words and 734562 word types
2021-07-14 13:43:08,325 - gensim.models.phrases - INFO - PROGRESS: at sentence #50000, processed 4582415 words and 842480 word types
2021-07-14 13:43:10,109 - gensim.models.phrases - INFO - PROGRESS: at sentence #60000, processed 5481494 words and 944597 w

CPU times: user 4min 38s, sys: 0 ns, total: 4min 38s
Wall time: 4min 37s


In [35]:
phrase_model.save(str(MODEL_DIR / 'phrases_gensim_lemmas_14_07_2021.pkl'))
print(len(text_corpus))
text_corpus[:10]

2021-07-14 15:04:00,144 - gensim.utils - INFO - Phrases lifecycle event {'fname_or_handle': '../models/phrases_gensim_lemmas_14_07_2021.pkl', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-07-14T15:04:00.144186', 'gensim': '4.0.1', 'python': '3.8.3 (default, May 19 2020, 18:47:26) \n[GCC 7.3.0]', 'platform': 'Linux-3.10.0-1160.25.1.el7.x86_64-x86_64-with-glibc2.10', 'event': 'saving'}
2021-07-14 15:04:00,144 - smart_open.smart_open_lib - DEBUG - {'uri': '../models/phrases_gensim_lemmas_14_07_2021.pkl', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'transport_params': None}


595371


[['хотеть_уточнить',
  'вчера',
  'карта',
  'первый_январь',
  'вечером',
  'списать',
  'деньга',
  'описание',
  'стоить',
  'сбербанк_онлайн',
  'скб_банк',
  'хотеть_узнать',
  'снять',
  'номер',
  'карта',
  'назвать_полностью',
  'секундочка',
  'фамилия_имя',
  'отчество_полностью',
  'надежда',
  'кодовый_слово',
  'помнить',
  'кодовый_слово',
  'ваш',
  'помнить',
  'сумма',
  'ещё',
  'уточнить',
  'сумма',
  'ещё',
  'уточнить',
  'сумма',
  'первый',
  'вечером',
  'час',
  'минута',
  'сайт',
  'видимо',
  'банк',
  'перевод',
  'перевод',
  'сайт',
  'банк',
  'дать',
  'карта',
  'указываться',
  'перевод',
  'проходить',
  'перевод',
  'никакой',
  'операция_совершать',
  'делать',
  'совершать_перевод',
  'понять',
  'случай',
  'действие',
  'ваш',
  'карта',
  'приостановить',
  'дальнейший_консультация',
  'соединить_специалист',
  'ожидание',
  'длительный',
  'удобно',
  'удобно',
  'слышать',
  'передать',
  'ваш',
  'вопрос',
  'дать',
  'ваш',
  'карта',
  '

2021-07-14 15:04:03,017 - gensim.utils - INFO - saved ../models/phrases_gensim_lemmas_14_07_2021.pkl


## Train Word2Vec

In [36]:
model = word2vec.Word2Vec(text_corpus, vector_size=200, hs=0, sg=0, negative=15, window=5, min_count=2,
                          workers=32, compute_loss=True)

2021-07-14 15:04:16,182 - gensim.models.word2vec - INFO - collecting all words and their counts
2021-07-14 15:04:16,182 - gensim.models.word2vec - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-07-14 15:04:16,425 - gensim.models.word2vec - INFO - PROGRESS: at sentence #10000, processed 790483 words, keeping 23529 word types
2021-07-14 15:04:16,668 - gensim.models.word2vec - INFO - PROGRESS: at sentence #20000, processed 1577951 words, keeping 33055 word types
2021-07-14 15:04:16,916 - gensim.models.word2vec - INFO - PROGRESS: at sentence #30000, processed 2358595 words, keeping 40277 word types
2021-07-14 15:04:17,161 - gensim.models.word2vec - INFO - PROGRESS: at sentence #40000, processed 3131812 words, keeping 46338 word types
2021-07-14 15:04:17,392 - gensim.models.word2vec - INFO - PROGRESS: at sentence #50000, processed 3868905 words, keeping 51582 word types
2021-07-14 15:04:17,629 - gensim.models.word2vec - INFO - PROGRESS: at sentence #60000, pro

In [37]:
model.wv.save_word2vec_format(MODEL_DIR / 'word2vec_gensim_lemmas_14_07_2021', binary=False)

2021-07-14 15:08:40,741 - gensim.models.keyedvectors - INFO - storing 91298x200 projection weights into ../models/word2vec_gensim_lemmas_14_07_2021
2021-07-14 15:08:40,741 - smart_open.smart_open_lib - DEBUG - {'uri': PosixPath('../models/word2vec_gensim_lemmas_14_07_2021'), 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'transport_params': None}


In [38]:
model.wv.key_to_index

{'карта': 0,
 'мочь': 1,
 'номер': 2,
 'сказать': 3,
 'ваш': 4,
 'минута': 5,
 'банк': 6,
 'сбербанк': 7,
 'ещё': 8,
 'понять': 9,
 'счёт': 10,
 'деньга': 11,
 'который': 12,
 'сумма': 13,
 'вопрос': 14,
 'хотеть': 15,
 'сбербанк_онлайн': 16,
 'информация': 17,
 'назвать': 18,
 'дать': 19,
 'сделать': 20,
 'операция': 21,
 'кредит': 22,
 'телефон': 23,
 'перевод': 24,
 'знать': 25,
 'тысяча': 26,
 'номер_телефон': 27,
 'рубль': 28,
 'получаться': 29,
 'говорить': 30,
 'день': 31,
 'заблокировать': 32,
 'звонить': 33,
 'сегодня': 34,
 'видеть': 35,
 'смотреть': 36,
 'проверить': 37,
 'свой': 38,
 'звать': 39,
 'уточнить': 40,
 'подсказать': 41,
 'посмотреть': 42,
 'должный': 43,
 'вообще': 44,
 'кодовый_слово': 45,
 'переводить': 46,
 'весь': 47,
 'платёж': 48,
 'правильно': 49,
 'перевести': 50,
 'смс': 51,
 'указать': 52,
 'последний_цифра': 53,
 'весь_свидание': 54,
 'делать': 55,
 'девушка': 56,
 'прийти': 57,
 'свидание': 58,
 'новый': 59,
 'ожидать': 60,
 'получить': 61,
 'оформит

In [39]:
model.wv.most_similar("списание", topn=10)

[('списание_произойти', 0.8068657517433167),
 ('произойти_списание', 0.7558993101119995),
 ('попытка_списание', 0.7522531151771545),
 ('списание_происходить', 0.7207934856414795),
 ('списать', 0.7035893797874451),
 ('сосание', 0.696039080619812),
 ('происходить_списание', 0.6897704601287842),
 ('списывание', 0.6739992499351501),
 ('списаться', 0.6423813700675964),
 ('запрос_списание', 0.6410989761352539)]

In [40]:
model.wv.most_similar("банкомат", topn=10)

[('банкомат_банкомат', 0.93598872423172),
 ('банкомат_терминал', 0.6965110301971436),
 ('вставлять_банкомат', 0.683822512626648),
 ('устройство_самообслуживание', 0.6589471697807312),
 ('возле_банкомат', 0.6206271648406982),
 ('банкомат_вставлять', 0.6202505230903625),
 ('банкомат_вставить', 0.6066932678222656),
 ('вставлять', 0.5977928638458252),
 ('терминал', 0.5967423915863037),
 ('терминал_терминал', 0.5761221647262573)]